This lab on K-Nearest Neighbors in R comes from p. 163-167 of "Introduction to Statistical Learning with Applications in R" by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani

# 4.6.5: K-Nearest Neighbors

In this lab, we will perform KNN on the ${\tt Smarket}$ dataset from ${\tt ISLR}$. This data set consists of percentage returns for the S&P 500 stock index over 1,250 days, from the
beginning of 2001 until the end of 2005. For each date, we have recorded
the percentage returns for each of the five previous trading days, ${\tt Lag1}$
through ${\tt Lag5}$. We have also recorded ${\tt Volume}$ (the number of shares traded on the previous day, in billions), ${\tt Today}$ (the percentage return on the date
in question) and ${\tt Direction}$ (whether the market was ${\tt Up}$ or ${\tt Down}$ on this
date). We can use the ${\tt head(...)}$ function to look at the first few rows:

In [ ]:
library(ISLR)
head(Smarket)

Today we're going to try to predict ${\tt Direction}$ using percentage returns from the previous two days (${\tt Lag1}$ and ${\tt Lag2}$). We'll build our model using the ${\tt knn()}$ function, which is part of the
${\tt class}$ library:

In [ ]:
library(class)

This function works rather differently from the other model-fitting
functions that we have encountered thus far. Rather than a two-step
approach in which we first fit the model and then we use the model to make
predictions, ${\tt knn()}$ forms predictions using a single command. The function
requires four inputs.
   1. A matrix containing the predictors associated with the training data,
labeled ${\tt train.X}$ below.
   2. A matrix containing the predictors associated with the data for which
we wish to make predictions, labeled ${\tt test.X}$ below.
   3. A vector containing the class labels for the training observations,
labeled ${\tt train.Direction}$ below.
   4. A value for $K$, the number of nearest neighbors to be used by the
classifier.

We'll first create a vector corresponding to the observations from 2001 through 2004, which we'll use to train the model. We will then use this vector to create a held out data set of observations from 2005 on which we will test.

In [ ]:
train=(Smarket$Year<2005)

The object ${\tt train}$ is a vector of 1,250 elements, corresponding to the observations
in our data set. The elements of the vector that correspond to
observations that occurred before 2005 are set to ${\tt TRUE}$, whereas those that
correspond to observations in 2005 are set to ${\tt FALSE}$. 

The object train is a **Boolean vector**, since its elements are ${\tt TRUE}$ and ${\tt FALSE}$. Boolean vectors can be used to obtain a subset of the rows or columns of a matrix. For
instance, the command ${\tt Smarket[train,]}$ would pick out a submatrix of the
stock market data set, corresponding only to the dates before 2005, since
those are the ones for which the elements of train are ${\tt TRUE}$:

In [ ]:
head(Smarket[train,])

The ${\tt !}$ symbol
can be used to reverse all of the elements of a Boolean vector. That is,
${\tt !train}$ is a vector similar to ${\tt train}$, except that the elements that are ${\tt TRUE}$
in train get swapped to ${\tt FALSE}$ in ${\tt !train}$, and the elements that are ${\tt FALSE}$
in train get swapped to ${\tt TRUE}$ in ${\tt !train}$. Therefore, ${\tt Smarket[!train,]}$ yields
a submatrix of the stock market data containing only the observations for
which train is ${\tt FALSE}$—that is, the observations with dates in 2005. The
output below indicates that there are 252 such observations:

In [ ]:
dim(Smarket[!train,])

Now we'll use the ${\tt cbind()}$ function, short for column bind, to bind the ${\tt Lag1}$ and ${\tt Lag2}$ variables together into two matrices, one for the training set and the
other for the test set. We'll also pull out our training labels.

In [ ]:
train.X=cbind(Smarket$Lag1,Smarket$Lag2)[train,]
test.X=cbind(Smarket$Lag1,Smarket$Lag2)[!train,]
train.Direction=Smarket$Direction[train]

Now the ${\tt knn()}$ function can be used to predict the market’s movement for
the dates in 2005. We set a **random seed** before we apply ${\tt knn()}$ because
if several observations are tied as nearest neighbors, then ${\tt R}$ will randomly
break the tie. Therefore, a seed must be set in order to ensure reproducibility
of results.

In [ ]:
set.seed(1)
knn.pred=knn(train.X,test.X,train.Direction,k=1)

The ${\tt table()}$ function can be used to produce a **confusion matrix** in order to determine how many observations were correctly or incorrectly classified.

In [ ]:
table(knn.pred,Smarket$Direction[!train])
mean(knn.pred==Smarket$Direction[!train])

The results using $K = 1$ are not very good, since only 50% of the observations
are correctly predicted. Of course, it may be that $K = 1$ results in an
overly flexible fit to the data. Below, we repeat the analysis using $K = 3$.

In [ ]:
knn.pred=knn(train.X,test.X,train.Direction,k=3)
table(knn.pred,Smarket$Direction[!train])
mean(knn.pred==Smarket$Direction[!train])

The results have improved slightly. Let's try a few other $K$ values to see if we get any further improvement:

In [ ]:
for (k_val in 1:10) {
    set.seed(1)
    # Your code here
}

It looks like for classifying this dataset, ${KNN}$ might not be the right approach.

# 4.6.6: An Application to Caravan Insurance Data
Let's see how the ${\tt KNN}$ approach performs on the ${\tt Caravan}$ data set, which is
part of the ${\tt ISLR}$ library. This data set includes 85 predictors that measure demographic characteristics for 5,822 individuals. The response variable is
${\tt Purchase}$, which indicates whether or not a given individual purchases a
caravan insurance policy. In this data set, only 6% of people purchased
caravan insurance.

In [ ]:
summary(Caravan$Purchase)

Because the ${\tt KNN}$ classifier predicts the class of a given test observation by
identifying the observations that are nearest to it, the scale of the variables
matters. Any variables that are on a large scale will have a much larger
effect on the distance between the observations, and hence on the ${\tt KNN}$
classifier, than variables that are on a small scale. 

For instance, imagine a
data set that contains two variables, salary and age (measured in dollars
and years, respectively). As far as ${\tt KNN}$ is concerned, a difference of \$1,000
in salary is enormous compared to a difference of 50 years in age. Consequently,
salary will drive the ${\tt KNN}$ classification results, and age will have
almost no effect. 

This is contrary to our intuition that a salary difference
of \$1,000 is quite small compared to an age difference of 50 years. Furthermore,
the importance of scale to the ${\tt KNN}$ classifier leads to another issue:
if we measured salary in Japanese yen, or if we measured age in minutes,
then we’d get quite different classification results from what we get if these
two variables are measured in dollars and years.

A good way to handle this problem is to **standardize** the data so that all
variables are given a mean of zero and a standard deviation of one. Then
all variables will be on a comparable scale. The ${\tt scale()}$ function does just
this. In standardizing the data, we exclude column 86, because that is the
qualitative ${\tt Purchase}$ variable.

In [ ]:
standardized.X=scale(Caravan[,-86])
var(Caravan[,1])
var(Caravan[,2])
var(standardized.X[,1])
var(standardized.X[,2])

Now every column of ${\tt standardized.X}$ has a standard deviation of one and
a mean of zero.

We'll now split the observations into a test set, containing the first 1,000
observations, and a training set, containing the remaining observations.

In [ ]:
test=1:1000
train2.X=standardized.X[-test,]
test2.X=standardized.X[test,]
train2.Y=Caravan$Purchase[-test]
test2.Y=Caravan$Purchase[test]

The vector test is numeric, with values from 1 through 1,000. Typing
${\tt standardized.X[test,]}$ yields the submatrix of the data containing the observations
whose indices range from 1 to 1, 000, whereas typing
${\tt standardized.X[-test,]}$ yields the submatrix containing the observations
whose indices do **not** range from 1 to 1, 000.

Let's fit a ${\tt KNN}$ model on the training data using $K = 1$, and evaluate its
performance on the test data.

In [ ]:
set.seed(1)
knn.pred=knn(train2.X,test2.X,train2.Y,k=1)
mean(test2.Y!=knn.pred) # KNN error rate
mean(test2.Y!="No")     # Percent of people who purchase insurance

The KNN error rate on the 1,000 test observations is just under 12%. At first glance, this may appear to be fairly good. However, since only 6% of customers purchased insurance, we could get the error rate down to 6% by always predicting ${\tt No}$ regardless of the values of the predictors!

Suppose that there is some non-trivial cost to trying to sell insurance
to a given individual. For instance, perhaps a salesperson must visit each
potential customer. If the company tries to sell insurance to a random
selection of customers, then the success rate will be only 6%, which may
be far too low given the costs involved. 

Instead, the company would like
to try to sell insurance only to customers who are likely to buy it. So the
overall error rate is not of interest. Instead, the fraction of individuals that
are correctly predicted to buy insurance is of interest.

It turns out that ${\tt KNN}$ with $K = 1$ does far better than random guessing
among the customers that are predicted to buy insurance:

In [ ]:
table(knn.pred,test2.Y)

Among 77 such
customers, 9, or 11.7%, actually do purchase insurance. This is double the
rate that one would obtain from random guessing. Let's see if increasing $K$ helps! Try out a few different $K$ values below. Feeling adventurous? Write a function that figures out the best value for $K$.

In [ ]:
set.seed(1)
# Your code here

It appears that ${\tt KNN}$ is finding some real patterns in a difficult data set! To get credit for this lab, post a response to the Piazza prompt available at: https://piazza.com/class/igwiv4w3ctb6rg?cid=10